In [7]:
import matplotlib.pyplot as plt
import cv2
# Technically not necessary in newest versions of jupyter
%matplotlib inline

In [8]:
# width,height,channels
image_shape = (224,224,3)
from keras.preprocessing.image import ImageDataGenerator

In [9]:
image_gen = ImageDataGenerator(rotation_range=30, # rotate the image 30 degrees
                               rescale=1/255, # Rescale the image by normalzing it.
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

# Creating the Model - CNN


In [10]:
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

In [11]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())


model.add(Dense(256))
model.add(Activation('relu'))

# Dropouts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of neurons.
model.add(Dropout(0.5))

# Last layer, remember its binary, 0=Benign , 1=Malignant
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [12]:
model.summary

<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x000001CC051067B8>>

### Training the Model

In [23]:
batch_size = 16

train_image_gen = image_gen.flow_from_directory('../skin-cancer-malignant-vs-benign/Data/train',
                                               target_size=image_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='binary')

Found 2637 images belonging to 2 classes.


In [24]:
test_image_gen = image_gen.flow_from_directory('../skin-cancer-malignant-vs-benign/Data/test',
                                                 target_size=image_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='binary')

Found 660 images belonging to 2 classes.


In [25]:
train_image_gen.class_indices

{'benign': 0, 'malignant': 1}

In [26]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
#the number of samples processed for each epoch is batch_size * steps_per_epochs
results = model.fit_generator(train_image_gen,epochs=100,
                              steps_per_epoch=50,
                              validation_data=test_image_gen,
                             validation_steps=12,
                             )

Epoch 1/100
50/50 [==============================] - 95s 2s/step - loss: 8.7882 - acc: 0.4487 - val_loss: 9.0340 - val_acc: 0.4333
Epoch 2/100
50/50 [==============================] - 94s 2s/step - loss: 8.6487 - acc: 0.4575 - val_loss: 9.4658 - val_acc: 0.4062
Epoch 3/100
50/50 [==============================] - 95s 2s/step - loss: 8.7882 - acc: 0.4487 - val_loss: 8.1373 - val_acc: 0.4896
Epoch 4/100
50/50 [==============================] - 94s 2s/step - loss: 8.8893 - acc: 0.4424 - val_loss: 8.3255 - val_acc: 0.4778
Epoch 5/100
50/50 [==============================] - 94s 2s/step - loss: 8.5690 - acc: 0.4625 - val_loss: 8.5524 - val_acc: 0.4635
Epoch 6/100
32/50 [==================>...........] - ETA: 32s - loss: 8.7496 - acc: 0.4512

KeyboardInterrupt: 

In [ ]:
import numpy as np
print(np.mean(results.history['acc']))
print(np.mean(results.history['val_acc']))
print(np.mean(results.history['loss']))
print(np.mean(results.history['val_loss']))


# Evaluating the Model

In [ ]:
plt.plot(results.history['acc'])
plt.plot(results.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Predicting on new images

In [ ]:
train_image_gen.class_indices

In [ ]:
import numpy as np
from keras.preprocessing import image

benign_file = '../skin-cancer-malignant-vs-benign/Data/train/'

benign_img = image.load_img(benign_file)

benign_img = image.img_to_array(dog_img)
benign_img = dog_img/255

In [ ]:
prediction_prob = model.predict(dog_img)

In [ ]:
# Output prediction
print(f'Probability that image is a dog is: {prediction_prob} ')